In [ ]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory_Q import ReplayMemory
from doubleq_networks import DoubleQ_Network
from test_game import test_env
from info_obj import info_holder
import os
import gym

#game_name = 'LunarLander-v2'
#game_name = 'CartPole-v0'
game_name = 'MountainCar-v0'
game = gym.make(game_name)
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
save_dir = './Results/'+ game_name[:-3] + '/net/'
string = ['hidden layers: ']
hid_lay = 8
with open(save_dir + 'params.txt', 'r') as f:
    xa = f.readline()
    while xa:
        if string[0] in xa:
            hid_lay = [int(xb) for xb in xa[len(string[0]):].split()]
        xa = f.readline()
print(hid_lay)
MAX_STEP = 1000
LEARNING_RATE = 0
TAU = 0
ENTROPY_BETA = 0
eps = 0
info = info_holder()
try:
    DQN
except:
    pass
else:
    DQN.reset_net()

graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.hid_layers = np.array(hid_lay)
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    DQN = DoubleQ_Network(info)

EP = 13000 - 1

with tf.Session(graph=graph) as sess:
    
    DQN.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    filename = save_dir + 'Episod_' + str(EP) + '.chk'
    saver.restore(sess, filename)
    ep_reward = np.zeros(100)
    for i in range(100):
        s = game.reset()
        a = DQN.take_action(s.reshape(-1,info.dim_s), eps)
        s2, r, terminal, report = game.step(a)
        
        for st in range(MAX_STEP):
            #game.render()
            a = DQN.take_action(s.reshape(-1,info.dim_s), eps)
            s2, r, terminal, report = game.step(a)
            s = s2
            ep_reward[i] += (1.*r)
            if terminal:
                #print('terminated')
                break
        #print(ep_reward[i])
        #pdb.set_trace()
    print('reward:%f' %np.mean(ep_reward))

In [ ]:
print(ep_reward)